In [1]:
import nltk
import numpy as np
from nltk.util import ngrams
import glob
import errno
import os
import string
import csv
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from sklearn.model_selection import train_test_split 
  
data=[]
N=3
K=1 #smothing
v_occur=1
exclude = set(string.punctuation)
exclude.union('\n')
stop_words = set(stopwords.words('english')) 
lemmatizer = WordNetLemmatizer() 


def convertTuple(tup): 
    str =  ''.join(tup) 
    return str

with open('Womens Clothing E-Commerce Reviews.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
         data.append(row)
d=np.array(data)
print(len(d))

#separate classes
d=d[1:,1:] #remove first line and column
l_recommendation=np.array(d[:,4])
title=np.array([d[:,2]])
review_text=np.array([d[:,3]])
review=np.concatenate((title.T,review_text.T),axis=1)

X_train, X_test, y_train, y_test = train_test_split(review, l_recommendation, test_size=0.2,random_state=0)
print(len(X_train), len(X_test), len(y_train), len(y_test))

print(list(l_recommendation).count('1'),list(l_recommendation).count('2'),list(l_recommendation).count('3'),list(l_recommendation).count('4'),list(l_recommendation).count('5'))
print(list(y_test).count('1'),list(y_test).count('2'),list(y_test).count('3'),list(y_test).count('4'),list(y_test).count('5'))
c_1=list(y_train).count('1')
c_2=list(y_train).count('2')
c_3=list(y_train).count('3')
c_4=list(y_train).count('4')
c_5=list(y_train).count('5')
c_total=c_1+c_2+c_3+c_4+c_5

review_1 = X_train[y_train == '1', :] # extract all rows with label 1
review_2 = X_train[y_train == '2', :] # extract all rows with label 2
review_3 = X_train[y_train == '3', :] # extract all rows with label 3
review_4 = X_train[y_train == '4', :] # extract all rows with label 4
review_5 = X_train[y_train == '5', :] # extract all rows with label 5


def ngram_text(review,N):
    text=""
    for i in range(len(review)):
        txt=str(review[i])
        s = ''.join(ch for ch in  txt if ch not in exclude)
        out=s.lower()
        result = re.sub(r'\d+', '', out) #remove numbers
        text=text+" "+result
        
    tokens = nltk.word_tokenize(text)
    lemmas= [lemmatizer.lemmatize(tokens[t]) for t in range(len(tokens))]
    filtered_sentence = [w for w in lemmas if not w in stop_words] 
    ungs = ngrams(filtered_sentence, N-1) #ngram-1
    ufdist = nltk.FreqDist(ungs)
    
    print("top 10 words u: ", ufdist.most_common(10))
    high_fdist = {convertTuple(k) for k, v in ufdist.items() if v >=v_occur} #remove tokens that occurs more than x times
    occurence_sentence = [w for w in filtered_sentence if w in high_fdist] 
    tokens_final=occurence_sentence
    
    ngs = ngrams(tokens_final, N) #Create n-grams
    nfdist = nltk.FreqDist(ngs)    
    
    print("top 10 words f: ", nfdist.most_common(10))
    vocabulary=len(nfdist)    
    prob=dict()
    for k,v in ufdist.items():
        prob[convertTuple(k)]=v
    
    return tokens_final, ngs, ungs, ufdist, nfdist, vocabulary, prob, high_fdist


tokens_1, ngs_1, ungs_1, ufdist_1, nfdist_1, vocabulary_1, prob_1, high_fdist_1=ngram_text(review_1,N)
print('vocab_1',vocabulary_1)
tokens_2, ngs_2, ungs_2, ufdist_2, nfdist_2, vocabulary_2, prob_2, high_fdist_2=ngram_text(review_2,N)
print('vocab_2',vocabulary_2)
tokens_3, ngs_3, ungs_3,ufdist_3, nfdist_3, vocabulary_3, prob_3, high_fdist_3=ngram_text(review_3,N)
print('vocab_3',vocabulary_3)
tokens_4, ngs_4, ungs_4, ufdist_4, nfdist_4, vocabulary_4, prob_4, high_fdist_4=ngram_text(review_4,N)
print('vocab_4',vocabulary_4)
tokens_5, ngs_5, ungs_5, ufdist_5, nfdist_5, vocabulary_5, prob_5, high_fdist_5=ngram_text(review_5,N)
print('vocab_5',vocabulary_5)
tokens_t, ngs_t, ungs_t, ufdist_t, nfdist_t, vocabulary_t, prob_t, high_fdist_t=ngram_text(X_train,N)
print('vocab_t',vocabulary_t)


inference=[]
for i in range(len(X_test)):  
    txt=str(X_test[i])
    s = ''.join(ch for ch in  txt if ch not in exclude)
    out=s.lower()
    result = re.sub(r'\d+', '', out) #remove numbers
    tokens =nltk.word_tokenize(result)
    lemmas= [lemmatizer.lemmatize(tokens[t]) for t in range(len(tokens))]
    filtered_sentence = [w for w in lemmas if not w in stop_words] 
    occurence_sentence = [w for w in filtered_sentence if w in high_fdist_t] 
    total_1=0
    total_2=0
    total_3=0
    total_4=0
    total_5=0
    for j in range(len(occurence_sentence)-N+1):
        condit_1=0
        condit_2=0
        condit_3=0
        condit_4=0
        condit_5=0
        
        if(N==4):
            thistuple3=(occurence_sentence[j],occurence_sentence[j+1],occurence_sentence[j+2],occurence_sentence[j+3])
            thistuple2=(occurence_sentence[j],occurence_sentence[j+1],occurence_sentence[j+2])
            condit_1=(nfdist_1.get(thistuple3,0)+K)/(ufdist_1.get(thistuple2,0)+(K*vocabulary_t))
            condit_2=(nfdist_2.get(thistuple3,0)+K)/(ufdist_2.get(thistuple2,0)+(K*vocabulary_t))
            condit_3=(nfdist_3.get(thistuple3,0)+K)/(ufdist_3.get(thistuple2,0)+(K*vocabulary_t))
            condit_4=(nfdist_4.get(thistuple3,0)+K)/(ufdist_4.get(thistuple2,0)+(K*vocabulary_t))
            condit_5=(nfdist_5.get(thistuple3,0)+K)/(ufdist_5.get(thistuple2,0)+(K*vocabulary_t))
        if(N==3):
            thistuple3=(occurence_sentence[j],occurence_sentence[j+1],occurence_sentence[j+2])
            thistuple2=(occurence_sentence[j],occurence_sentence[j+1])
            if nfdist_t.get(thistuple3,0)>0:
                condit_1=(nfdist_1.get(thistuple3,0)+K)/(ufdist_1.get(thistuple2,0)+(K*vocabulary_t))
                condit_2=(nfdist_2.get(thistuple3,0)+K)/(ufdist_2.get(thistuple2,0)+(K*vocabulary_t))
                condit_3=(nfdist_3.get(thistuple3,0)+K)/(ufdist_3.get(thistuple2,0)+(K*vocabulary_t))
                condit_4=(nfdist_4.get(thistuple3,0)+K)/(ufdist_4.get(thistuple2,0)+(K*vocabulary_t))
                condit_5=(nfdist_5.get(thistuple3,0)+K)/(ufdist_5.get(thistuple2,0)+(K*vocabulary_t))           
        if(N==2):
            thistuple3=(occurence_sentence[j],occurence_sentence[j+1])
            thistuple2=(occurence_sentence[j])
            if nfdist_t.get(thistuple3,0)>0:
                condit_1=(nfdist_1.get(thistuple3,0)+K)/(prob_1.get(thistuple2,0)+(K*vocabulary_t))
                condit_2=(nfdist_2.get(thistuple3,0)+K)/(prob_2.get(thistuple2,0)+(K*vocabulary_t))
                condit_3=(nfdist_3.get(thistuple3,0)+K)/(prob_3.get(thistuple2,0)+(K*vocabulary_t))
                condit_4=(nfdist_4.get(thistuple3,0)+K)/(prob_4.get(thistuple2,0)+(K*vocabulary_t))
                condit_5=(nfdist_5.get(thistuple3,0)+K)/(prob_5.get(thistuple2,0)+(K*vocabulary_t))
        
        if(N==1):
            thistuple2=occurence_sentence[j]
            if prob_t.get(occurence_sentence[j],0)>0:
                condit_1=(prob_1.get(occurence_sentence[j],0)+K)/(len(tokens_1)+(K*vocabulary_t))
                condit_2=(prob_2.get(occurence_sentence[j],0)+K)/(len(tokens_2)+(K*vocabulary_t))
                condit_3=(prob_3.get(occurence_sentence[j],0)+K)/(len(tokens_3)+(K*vocabulary_t))
                condit_4=(prob_4.get(occurence_sentence[j],0)+K)/(len(tokens_4)+(K*vocabulary_t))
                condit_5=(prob_5.get(occurence_sentence[j],0)+K)/(len(tokens_5)+(K*vocabulary_t))
        
        if condit_1>0:
            total_1+=np.log2(condit_1)
        if condit_2>0:
            total_2+=np.log2(condit_2)
        if condit_3>0:
            total_3+=np.log2(condit_3)
        if condit_4>0:
            total_4+=np.log2(condit_4)
        if condit_5>0:
            total_5+=np.log2(condit_5)
    
    total_1_fim=np.log2(c_1/c_total)+total_1
    total_2_fim=np.log2(c_2/c_total)+total_2
    total_3_fim=np.log2(c_3/c_total)+total_3
    total_4_fim=np.log2(c_4/c_total)+total_4
    total_5_fim=np.log2(c_5/c_total)+total_5
    get_max=max(total_1_fim, total_2_fim, total_3_fim, total_4_fim, total_5_fim) 
    
    if(get_max==total_1_fim):
        inference.append('1')
    elif(get_max==total_2_fim):
        inference.append('2')
    elif(get_max==total_3_fim):
        inference.append('3') 
    elif(get_max==total_4_fim):
        inference.append('4')
    elif(get_max==total_5_fim):
        inference.append('5')
    else:
        inference.append('0')
        print(i,total_1_fim, total_2_fim, total_3_fim, total_4_fim, total_5_fim) 
    


print(len(inference), len(y_test))       
accuracy=0
errors=[]
for i in range(len(inference)):
    if inference[i]==y_test[i]:
                         accuracy+=1
    else:
        errors.append(i)

print(len(errors))
print(accuracy/len(inference))


23487
18788 4698 18788 4698
842 1565 2871 5077 13131
175 327 566 1007 2623
top 10 words u:  [(('look', 'like'), 64), (('going', 'back'), 36), (('looked', 'like'), 35), (('wa', 'excited'), 33), (('poor', 'quality'), 32), (('wanted', 'love'), 25), (('wa', 'really'), 23), (('nothing', 'like'), 23), (('dress', 'wa'), 22), (('wa', 'disappointed'), 22)]
top 10 words f:  [(('look', 'nothing', 'like'), 11), (('wa', 'really', 'excited'), 8), (('really', 'wanted', 'love'), 7), (('like', 'wa', 'wearing'), 6), (('wa', 'excited', 'get'), 5), (('really', 'wanted', 'like'), 5), (('first', 'time', 'wore'), 5), (('look', 'cute', 'model'), 5), (('wa', 'excited', 'order'), 5), (('like', 'maternity', 'top'), 5)]
vocab_1 19549
top 10 words u:  [(('wanted', 'love'), 106), (('look', 'like'), 96), (('looked', 'like'), 83), (('going', 'back'), 80), (('really', 'wanted'), 63), (('made', 'look'), 53), (('dress', 'wa'), 53), (('wa', 'excited'), 51), (('didnt', 'work'), 49), (('wa', 'disappointed'), 49)]
top 10 wo